#### Overview

Part 1: Basic text analysis

Part 2: Cleaning and normalization

Ideas for complex/advanced applications

### Part 1: Basic Text Analysis 

NLTK (Natural Language Toolkit): 

A python library with functions specifically designed to analyze natural (not-computer) language.

NLTK needs to be imported in any python script that uses it


In [ ]:
import nltk
from urllib.request import urlopen


NLTK comes with two built-in texts, *Moby Dick* and *Sense and Sensibility.* Note the assumptions about how NLTK will be used: novels, long, canonical...

#### **NLTK functions: Concordance()**

- Called on a Text object and takes a string (a sequence of characters) as an argument

- text_variable.condordance(string_arg) 

- Calling condordance("word") returns the words that surround “word” in different sentences, helping us to get a glimpse of the contexts in which the word shows up. For example:

In [ ]:
my_url = "https://raw.githubusercontent.com/ucla/ca-dhri/main/Day2/Liberator91901.txt"

file = urlopen(my_url)
liberator_raw = file.read()
liberator_txt = liberator_raw.decode()
txt_tokens = nltk.word_tokenize(liberator_txt)
liberator = nltk.Text(txt_tokens)

#let's make sure we created a text object
liberator

In [ ]:
#call concordance on the word "law"
liberator.concordance("law")



#### **NLTK functions: Similar()**

-text.similar(string_arg)

-Like concordance, similar will find the contexts of the string variable it is given, but it can also compare the content of these contexts to all other words, looking for words that are used in similar contexts to the given argument

In [ ]:
liberator.similar("law")

#### **NLTK functions: dispersion plot**

-text.dispersion_plot(list_arg)

-Takes a list of strings as input (not a single string!) and outputs a graph of the instances where each word appears. If you want to make a plot for one string, pass the function a one-object list [“example”]

-Note: dispersion_plot() is helpful for seeing how language changes over time or over narrative arcs. It might be more useful on a large collection of newspapers over time than on a single newspaper.


In [ ]:
liberator.dispersion_plot(["law"])

In [ ]:
liberator.dispersion_plot(["law","furniture","Angeles"])

#### **NLTK functions: count()**

-Takes a word as an argument and returns a count of each instance of that word in a text. 

-It is case sensitive (we will address cases in data cleaning)

In [ ]:
liberator.count("Angeles")

In [ ]:
liberator.count("angeles")

### Practice in Breakout Rooms

Can you think of how we could generate a concordance that would allow us to extract addresses from the text? What might we do with that information?

#### **Python operations on NLTK objects: len(), set()**

So far, we have been using the built-in NLTK corpus to analyze our text object. We can also use regular python expressions on it:

**len(text_object)** returns the length of the nltk object, that is, the number of words in the text. In a pre-cleaning text, this will include punctuation and metadata. 

**set(text_object)** creates a set (a list without duplicates) of all the unique words 

**len(set(text_object)** returns — you guessed it — the length of the set of unique words, ie, the number of unique words in the text.

In [ ]:
len(liberator)

In [ ]:
set(liberator)
sorted(set(liberator))[:30]

In [ ]:
len(set(liberator))

### Part 2: Cleaning and Normalizing Data

#### **Removing capitalization and punctuation**

Type vs. token: 
- angeles vs. Angeles vs. ANGELES are distinct types
    
- A token is an instance of a type

- nltk.count(“angeles”) counts the number of tokens of that type

If the distinction between cases isn't important in your analysis, making all values of a text lowercase can be useful.

So we'll start normalizing the text by making all words lower case

In [ ]:
liberator_lowercase = [word.lower() for word in liberator] 

In [ ]:
liberator_lowercase.count("angeles")

#### **Remove all punctuation**

Lets run this: 

In [ ]:
liberator_lowercase_textonly = [word.lower() for word in liberator if word.isalpha()]

In [ ]:
#What did we just do, though?
#liberator_lowercase_textonly = [word.lower() for word in liberator if word.isalpha()] is shorthand for this function

liberator_lowercase_textonly = []				#define an empty list called liberator_lowercase_only

for w in liberator:					        #For each word ("w") in our existing text object 

	if w.isalpha():				        #if the word (“w”) is letters (not punctuation)
        
		liberator_lowercase_textonly.append(w.lower())  	#make it lowercase and add to our new list

        #if the word is not alpha, the for loop will move on to the next word 


TIP: be smart about your variable names 

#### **Removing stop words**

- ("the," "an," "a," etc.)


In [ ]:
#a predefined list of stopwords, how nice!
nltk.download('stopwords')
from nltk.corpus import stopwords

#note, stopwords' type isn't a List, it's a WordListCorpusReader
print(type(stopwords))

#if we want a "real" list, we need to call the words attribute
print(type(stopwords.words()))
      
#here's what we have:
print(stopwords.words('english'))

In [ ]:
#Get rid of stopwords in our text

#make sure you import stopwords somewhere in the script before you call it
liberator_sans_stops = []
stops = stopwords.words('english')

#define a new list
for word in liberator_lowercase_textonly:				#for each word in our cleaner list,
    if word not in stops:	                        #if that word isn’t in the list of stopwords,
        liberator_sans_stops.append(word)		        #add it to our new list


In [ ]:
#did it work?
liberator_sans_stops.count("angeles")

In [ ]:
liberator_sans_stops.count("of")

In [ ]:
liberator_sans_stops.count("after")

#### **Lemmatizing words**

- Lemmatization shrinks words to their grammatical root
    - example, cats ⭢ cat and walked ⭢ walk
  
- This gets complicated in the case of men ⭢ man and sang ⭢ sing 

- Lemmatization looks up a word in a reference dictionary and finds the appropriate root (though this still is not entirely accurate and takes a long time, since each word must be looked up in a reference)

- NLTK comes with pre-built stemmers and lemmatizers. 

In [ ]:
from nltk.stem import WordNetLemmatizer	

#create an instance of it for our function
wordnet_lemmatizer = WordNetLemmatizer()	

print(wordnet_lemmatizer.lemmatize("children"))

print(wordnet_lemmatizer.lemmatize("better"))

#for a word like “better,” need to specify grammatical function
print(wordnet_lemmatizer.lemmatize("better", pos='a'))
print(wordnet_lemmatizer.lemmatize("better", pos='n'))

#Parts of Speech
#ADJ, ADJ_SAT, ADV, NOUN, VERB = "a", "s", "r", "n", "v"

liberator_lemma = []
for word in liberator_sans_stops:
    word_lem = wordnet_lemmatizer.lemmatize(word)
    liberator_lemma.append(word_lem)   

### Part 3: Basic text analysis with our clean text

Because we've lemmatized our text, the results of functions like concordance, similar, and count will change. 

In [ ]:
#we have to make our clean text an NLTK object to use the functions
liberator_clean = nltk.Text(liberator_lemma)

liberator_clean.concordance('law')

In [ ]:
liberator_clean.concordance("children")

In [ ]:
liberator_clean.concordance("child")

In [ ]:
liberator_clean.concordance("anarchist")

In [ ]:
liberator_clean.concordance("lawless")

In [ ]:
liberator_clean.similar("good")

In [ ]:
liberator_clean.similar("child")

In [ ]:
liberator_clean.dispersion_plot(["law","america","anarchist","assassin",])

In [ ]:
liberator.dispersion_plot(["law","America","anarchist","assassin",])